<a href="https://colab.research.google.com/github/anahigarnelo/medication_monitoring/blob/main/interview_08202024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.8 MB/s eta 0:00:00


1. Lamdba function with missing logic
  * Adherence Status:
      * Adherent: When the total dosage taken by the patient matches the prescribed dosage.
      * Non-Adherent: When the total dosage taken by the patient is less than the prescribed dosage.
      * Overdose: When the total dosage taken by the patient is more than the prescribed dosage.
  * Delayed Doses:
    * Increment Delayed Doses: Track the number of times a patient took their medication late. If the schedule_adherence is marked as "Late", increment the count.
  * Storing Doctor Statistics*(Optional)*:
    * Doctor Statistics: Track and store statistics about doctors, such as the total dosage they've prescribed and the number of patients they are treating. This section is optional and can be implemented if needed.

In [ ]:
import json
import boto3
from collections import defaultdict

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('MedicationData')

def lambda_handler(event, context):
    # Get the patient_id from the event, if provided
    patient_id_to_query = event.get('patient_id', None)

    # Hardcoded data instead of reading from a CSV file
    medication_data = [
        {
            'patient_id': 'P001',
            'dosage': 150,
            'medication': 'Aspirin',
            'administered_at': '2024-08-01 09:00:00',
            'prescribed_dosage': 200,
            'prescribing_doctor': 'Dr. Smith',
            'schedule_adherence': 'On Time'
        },
        {
            'patient_id': 'P002',
            'dosage': 100,
            'medication': 'Ibuprofen',
            'administered_at': '2024-08-01 10:00:00',
            'prescribed_dosage': 100,
            'prescribing_doctor': 'Dr. Jones',
            'schedule_adherence': 'Late'
        },
        {
            'patient_id': 'P001',
            'dosage': 50,
            'medication': 'Paracetamol',
            'administered_at': '2024-08-01 11:00:00',
            'prescribed_dosage': 50,
            'prescribing_doctor': 'Dr. Smith',
            'schedule_adherence': 'Late'
        },
        {
            'patient_id': 'P003',
            'dosage': 200,
            'medication': 'Aspirin',
            'administered_at': '2024-08-01 12:00:00',
            'prescribed_dosage': 200,
            'prescribing_doctor': 'Dr. Taylor',
            'schedule_adherence': 'On Time'
        },
    ]

    # Initialize dictionaries for processing
    total_dosage = defaultdict(int)
    adherence = {}
    delayed_doses = defaultdict(int)
    doctor_statistics = defaultdict(lambda: {'patients': set(), 'total_dosage': 0})

    # Processing data
    for entry in medication_data:
        patient_id = entry['patient_id']
        dosage = entry['dosage']
        prescribed_dosage = entry['prescribed_dosage']
        schedule_adherence = entry['schedule_adherence']
        doctor_id = entry['prescribing_doctor']

        total_dosage[patient_id] += dosage

        # Adherence Status Logic
        # Missing logic to determine adherence status

        # Delayed Doses Logic
        # Missing logic to track delayed doses

        # Track doctor statistics
        # Missing logic to track doctor statistics

    # If a specific patient_id was requested, return only that patient's data
    if patient_id_to_query:
        if patient_id_to_query in adherence:
            return {
                'statusCode': 200,
                'body': json.dumps({
                    'patient_id': patient_id_to_query,
                    'adherence_status': adherence[patient_id_to_query],
                    'total_dosage': total_dosage[patient_id_to_query],
                    'delayed_doses': delayed_doses[patient_id_to_query]
                })
            }
        else:
            return {
                'statusCode': 404,
                'body': json.dumps({
                    'error': f"Patient ID {patient_id_to_query} not found."
                })
            }

    # If no specific patient_id was requested, return data for all patients
    all_patients_data = []
    for patient_id in adherence:
        all_patients_data.append({
            'patient_id': patient_id,
            'adherence_status': adherence[patient_id],
            'total_dosage': total_dosage[patient_id],
            'delayed_doses': delayed_doses[patient_id]
        })

    return {
        'statusCode': 200,
        'body': json.dumps(all_patients_data)
    }

def store_in_dynamodb(patient_id, adherence_status, delayed_doses):
    try:
        table.put_item(
            Item={
                'patient_id': patient_id,
                'adherence_status': adherence_status,
                'delayed_doses': delayed_doses
            }
        )
    except Exception as e:
        print(f"Error storing data in DynamoDB: {str(e)}")

def store_doctor_statistics(doctor_id, stats):
    try:
        table.put_item(
            Item={
                'doctor_id': doctor_id,
                'total_dosage': stats['total_dosage'],
                'patient_count': len(stats['patients'])
            }
        )
    except Exception as e:
        print(f"Error storing doctor statistics in DynamoDB: {str(e)}")


In [ ]:
# Mock event and context for local testing
event = {
    "patient_id": "P001"
}
context = {}

# Run the Lambda function locally
response = lambda_handler(event, context)
print(json.dumps(response, indent=4))

{
    "statusCode": 404,
    "body": "{\"error\": \"Patient ID P001 not found.\"}"
}


2. What unit tests would you write to test out functionality?

3. How would you refactor the following aspects of the code to make it more efficient?


4. How would you improve the Lambda function itself to handle larger datasets efficiently and scale as needed?

5. Backup that can run on https://onecompiler.com/python

In [ ]:
import json
from collections import defaultdict

def lambda_handler(event, context):
    # Get the patient_id from the event, if provided
    patient_id_to_query = event.get('patient_id', None)

    # Hardcoded data instead of reading from a CSV file
    medication_data = [
        {
            'patient_id': 'P001',
            'dosage': 150,
            'medication': 'Aspirin',
            'administered_at': '2024-08-01 09:00:00',
            'prescribed_dosage': 200,
            'prescribing_doctor': 'Dr. Smith',
            'schedule_adherence': 'On Time'
        },
        {
            'patient_id': 'P002',
            'dosage': 100,
            'medication': 'Ibuprofen',
            'administered_at': '2024-08-01 10:00:00',
            'prescribed_dosage': 100,
            'prescribing_doctor': 'Dr. Jones',
            'schedule_adherence': 'Late'
        },
        {
            'patient_id': 'P001',
            'dosage': 50,
            'medication': 'Paracetamol',
            'administered_at': '2024-08-01 11:00:00',
            'prescribed_dosage': 50,
            'prescribing_doctor': 'Dr. Smith',
            'schedule_adherence': 'Late'
        },
        {
            'patient_id': 'P003',
            'dosage': 200,
            'medication': 'Aspirin',
            'administered_at': '2024-08-01 12:00:00',
            'prescribed_dosage': 200,
            'prescribing_doctor': 'Dr. Taylor',
            'schedule_adherence': 'On Time'
        },
    ]

    # Initialize dictionaries for processing
    total_dosage = defaultdict(int)
    adherence = {}
    delayed_doses = defaultdict(int)
    doctor_statistics = defaultdict(lambda: {'patients': set(), 'total_dosage': 0})

    # Processing data
    for entry in medication_data:
        patient_id = entry['patient_id']
        dosage = entry['dosage']
        prescribed_dosage = entry['prescribed_dosage']
        schedule_adherence = entry['schedule_adherence']
        doctor_id = entry['prescribing_doctor']

        total_dosage[patient_id] += dosage

        # Adherence Status Logic
        # Missing logic to determine adherence status

        # Delayed Doses Logic
        # Missing logic to track delayed doses

        # Track doctor statistics
        # Missing logic to track doctor statistics

    # If a specific patient_id was requested, return only that patient's data
    if patient_id_to_query:
        if patient_id_to_query in adherence:
            return {
                'statusCode': 200,
                'body': json.dumps({
                    'patient_id': patient_id_to_query,
                    'adherence_status': adherence[patient_id_to_query],
                    'total_dosage': total_dosage[patient_id_to_query],
                    'delayed_doses': delayed_doses[patient_id_to_query]
                })
            }
        else:
            return {
                'statusCode': 404,
                'body': json.dumps({
                    'error': f"Patient ID {patient_id_to_query} not found."
                })
            }

    # If no specific patient_id was requested, return data for all patients
    all_patients_data = []
    for patient_id in adherence:
        all_patients_data.append({
            'patient_id': patient_id,
            'adherence_status': adherence[patient_id],
            'total_dosage': total_dosage[patient_id],
            'delayed_doses': delayed_doses[patient_id]
        })

    return {
        'statusCode': 200,
        'body': json.dumps(all_patients_data)
    }


In [ ]:
# Mock event and context for local testing
event = {
    "patient_id": "P001"
}
context = {}

# Run the Lambda function locally
response = lambda_handler(event, context)
print(json.dumps(response, indent=4))

{
    "statusCode": 404,
    "body": "{\"error\": \"Patient ID P001 not found.\"}"
}
